# Module 13: Create server-side programming constructs in Azure Cosmos DB SQL API

- [[Learning path]](https://docs.microsoft.com/en-us/learn/paths/create-server-side-programming-azure-cosmos-db-sql-api/?ns-enrollment-type=Collection&ns-enrollment-id=1k8wcz8zooj2nx)
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/32-create-sproc-portal.html): Create a stored procedure with the Azure portal
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/33-create-use-udf-sdk.html): Implement and then use a UDF using the SDK

## Demo setup

In [ ]:
Connect-AzAccount
Set-AzContext -Subscription "b895a719-7034-411a-9944-ff196d1f450f"
$connectionString = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "ConnectionStrings")["Primary SQL Connection String"]
$primaryMasterKey = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "Keys")["PrimaryMasterKey"]
$documentEndpoint = (Get-AzCosmosDBAccount -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned).DocumentEndpoint

In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: Microsoft.Azure.Cosmos , 3.22.1"

#!share --from pwsh connectionString
#!share --from pwsh primaryMasterKey
#!share --from pwsh documentEndpoint

public class Product 
{ 
    public string id { get; set; }
    public string name { get; set; }
    public string categoryId { get; set; } // Assume categoryId is the partition key
    public string categoryName { get; set; }
    public double price { get; set; }
    public string[] tags { get; set; } 
}

In [ ]:
cosmicworks --endpoint $documentEndpoint --key $primaryMasterKey --datasets product

## Build multi-item transactions with the Azure Cosmos DB SQL API

### Understand transactions

In a database, a transaction is typically defined as a sequence of point operations grouped together into a single unit of work. It's expected that a transaction provides ACID guarantees.

- **Atomicity** guarantees that all the work done inside a transaction is treated as a single unit where either all of it is committed or none.
- **Consistency** makes sure that the data is always in a healthy internal state across transactions.
- **Isolation** guarantees that no two transactions interfere with each other – generally, most commercial systems provide multiple isolation levels that can be used based on the application's needs.
- **Durability** ensures that any change that's committed in the database will always be present.

Stored procedures are scoped to a single logical partition. You cannot execute a stored procedure that performs operations across logical partition key values.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/build-multi-item-transactions-azure-cosmos-db-sql-api/media/2-transaction.png)

For long-running lists of operations, a helper boolean value is returned by any JavaScript function that performs an operation indicating whether that operation is expected to complete within the request timeout duration. 

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/build-multi-item-transactions-azure-cosmos-db-sql-api/media/2-continuation.png)

### Author Stored procedures

```javascript
function createProduct(item) {
    var context = getContext();
    var container = context.getCollection(); 
    var accepted = container.createDocument(
        container.getSelfLink(),
        item,
        (error, newItem) => {
            if (error) throw error;
            context.getResponse().setBody(newItem)
        }
    );
    if (!accepted) return;
}
```

### Rollback transactions

Azure Cosmos DB’s SQL API will roll back the entire transaction if a single exception is thrown from the strored procedure script.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/build-multi-item-transactions-azure-cosmos-db-sql-api/media/4-rollback.png)

### Create stored procedures with the SDK

In [ ]:
using System;
using System.Linq;
using Microsoft.Azure.Cosmos;
using Microsoft.Azure.Cosmos.Scripts;

StoredProcedureProperties properties = new()
{
    Id = "createProduct",
    Body = @"
    function createProduct(title) {
        var context = getContext();
        var container = context.getCollection();
        var doc = {
          name: title,
          categoryId: 'demo'
        }
        var accepted = container.createDocument(
          container.getSelfLink(),
          doc,
          (error, newDoc) => {
            if (error) throw new Error(error.message);
            context.getResponse().setBody(newDoc);
          }
        );
        if (!accepted) return;
      }
    "
};

CosmosClient client = new (connectionString);
Database database = client.GetDatabase("cosmicworks");
Container container = database.GetContainer("products");

await container.Scripts.CreateStoredProcedureAsync(properties);

var result = await container.Scripts.ExecuteStoredProcedureAsync<dynamic>("createProduct", new PartitionKey("demo"), new dynamic[] {"Bike"});

result

Check with following SQL Query if item was created:

```sql
SELECT * FROM c
where c.name = "Bike"
```

## Expand query and transaction functionality in Azure Cosmos DB SQL API

### Create User-defined functions (UDFs)

UDFs are used to extend the Azure Cosmos DB SQL API’s query language grammar and implement custom business logic and can only be called from inside queries.

Suppose you have the following item

```json
{ 
  "name": "Black Bib Shorts (Small)", 
  "price": 80.00 
}
```

Create a UDF that calculates 15% tax

```javascript
function addTax(preTax) 
{ 
    return preTax * 1.15; 
}
```

Run this query that returns the price and the tax

```sql
SELECT 
    p.name, 
    p.price, 
    udf.addTax(p.price) AS priceWithTax 
FROM products p
WHERE p.name = "Black Bib Shorts (Small)“
```

### Create User-defined functions (UDFs) with the SDK

In [ ]:
using System;
using System.Linq;
using Microsoft.Azure.Cosmos;
using Microsoft.Azure.Cosmos.Scripts;

UserDefinedFunctionProperties udf = new () {
  Id = "addTax",
  Body = @"
    function addTax(preTax) {
      return preTax * 1.15;
    }"
};

CosmosClient client = new (connectionString);
Database database = client.GetDatabase("cosmicworks");
Container container = database.GetContainer("products");

await container.Scripts.CreateUserDefinedFunctionAsync(udf);

In [ ]:
QueryDefinition query = new ("SELECT TOP 10 c.name, c.price, udf.addTax(c.price) AS priceWithTax FROM c");

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var record in currentResultSet)
    {
        Console.WriteLine($"[{record.name,40}]\t{record.price,15:C}\t{record.priceWithTax,15:C}");
    }
}

### Add triggers to an operation – Pre-trigger

Pre-triggers are the core way that Azure Cosmos DB SQL API can inject business logic before an operations and cannot have any input parameters.

Suppose you want to insert the following item

```json
{ 
  "id": "caab0e5e-c037-48a4-a760-140497d19452", 
  "name": "Handlebar", 
  "categoryId": "e89a34d2-47ee-4da8-bcf6-10f552604b79",
  "categoryName": "Accessories", 
  "price": 50
}
```

You want to automatically insert a "label" so that the document becomes

```json
{ 
  "id": "caab0e5e-c037-48a4-a760-140497d19452", 
  "name": "Handlebar", 
  "categoryId": "e89a34d2-47ee-4da8-bcf6-10f552604b79",
  "categoryName": "Accessories", 
  "price": 50, 
  "label": "new"
}
```

Define the following pre-trigger

```javascript
function addLabel(item) 
{ 
    var context = getContext(); 
    var request = context.getRequest(); 
    var pendingItem = request.getBody(); 

    if (!('label' in pendingItem)) 
        pendingItem['label'] = 'new’; 

    request.setBody(pendingItem); 
}
```

### Add triggers to an operation – Post-trigger

Post-triggers are the core way that Azure Cosmos DB SQL API can inject business logic after an operations completes and if needed, can have any input parameters.

Define the following post-trigger

```javascript
function createView() 
{ 
    var context = getContext(); 
    var container = context.getCollection(); 
    var response = context.getResponse(); 
    var createdItem = response.getBody(); 

    var viewItem = { 
        sourceId: createdItem.id,
        categoryId: createdItem.categoryId, 
        displayName: `${createdItem.name} [${createdItem.categoryName}]` 
    };
 
    var accepted = container.createDocument( 
        container.getSelfLink(), 
        viewItem, (
            error, newItem) => { if (error) throw error; } 
        ); 

    if (!accepted) return; 
}
```

When you insert the following item

```json
{ 
  "id": "caab0e5e-c037-48a4-a760-140497d19452", 
  "name": "Handlebar", 
  "categoryId": "e89a34d2-47ee-4da8-bcf6-10f552604b79",
  "categoryName": "Accessories", 
  "price": 50
}
```

The post-trigger will create this additional item

```json
{ 
    "sourceId": "caab0e5e-c037-48a4-a760-140497d19452", 
    "categoryId": "e89a34d2-47ee-4da8-bcf6-10f552604b79", 
    "displayName": "Handlebar [Accessories]"
}
```

### Create triggers with the SDK

The Scripts property in the Microsoft.Azure.Cosmos.Container class contains a CreateTriggerAsync method that is used to create a new pre/post trigger from code.

Define a Pre-trigger

In [ ]:
string preTrigger = 
@"function addLabel() { 
    var context = getContext(); 
    var request = context.getRequest();
 
    var pendingItem = request.getBody(); 

    if (!('label' in pendingItem)) 
        pendingItem['label'] = 'new'; 

    request.setBody(pendingItem); 
}";

TriggerProperties properties = new() 
{ 
    Id = "addLabel", 
    Body = preTrigger, 
    TriggerOperation = TriggerOperation.Create, 
    TriggerType = TriggerType.Pre 
};
await container.Scripts.CreateTriggerAsync(properties);

Define a Post-trigger

In [ ]:
string postTrigger = 
@"function createView() { 
    var context = getContext(); 
    var container = context.getCollection(); 
    var response = context.getResponse(); 
    var createdItem = response.getBody(); 
    var viewItem = { 
        sourceId: createdItem.id, 
        categoryId: createdItem.categoryId, 
        displayName: `${createdItem.name} [${createdItem.categoryName}]` };
    var accepted = container.createDocument( 
        container.getSelfLink(), 
        viewItem, (error, newItem) => { if (error) throw error; } ); 
    if (!accepted) return; 
}";

TriggerProperties properties = new() 
{ 
    Id = "createView", 
    Body = postTrigger, 
    TriggerOperation = TriggerOperation.Create, 
    TriggerType = TriggerType.Post
};

await container.Scripts.CreateTriggerAsync(properties);


### Use a trigger in an operation with the SDK

When the triggers have been defined and created within the container, you can use them in an operation on the same container.

In [ ]:
ItemRequestOptions options = new() 
{ 
    PreTriggers = new List<string> { "addLabel" }, 
    PostTriggers = new List<string> { "createView" } 
};

var id = Guid.NewGuid().ToString();

Product saddle = new() 
{ 
    id = id, 
    categoryId = "26C74104-40BC-4541-8EF5-9892F7F03D72", 
    name = "LL Road Seat/Saddle", 
    categoryName = "Horses",
    price = 27.12d, 
    tags = new string[] { "brown", "weathered" } 
};

await container.CreateItemAsync(saddle, requestOptions: options);

id

Validate if the document LABEL has been added:

```sql
SELECT * FROM c where c.id = "97f5e49f-9beb-4e2a-af84-eae7a5cf1fc3"
```

Validate that an additional document has been created (post trigger "createView"). It should contain a displayName

```sql
SELECT * FROM c where c.sourceId = "97f5e49f-9beb-4e2a-af84-eae7a5cf1fc3"
```